# Filter Cities with ≥80% Valid Blocks (QC GPKGs)

This notebook:

- Reads RF prediction GPKGs: `predictions/{country}_rf_preds.gpkg`
- For each **country**:
  - Computes, per city, the share of blocks with valid `rf_label` and `POP_SEG`
  - Keeps only cities where **PctValid ≥ 80%**
  - Within those cities, keeps only blocks with valid `rf_label` and `POP_SEG`
  - Saves a filtered GPKG with **all original attributes** preserved

It also writes a **summary CSV** with basic counts and population before/after filtering.

> **Note:** In the analysis behind the manuscript, *all cities passed the 80% QC check*, so the filtered GPKGs are effectively identical to the original prediction files.  
> The code is included here for full transparency, so others can reproduce and verify the QC step.
  
**Inputs**

- RF predictions (per-country GPKG):  
  `2_modelling/02_application/predictions/{country}_rf_preds.gpkg`  
  Required columns: `REG1_GHSL`, `UC_NM_MN`, `rf_label`, `POP_SEG`, `geometry`.

**Outputs**

- Filtered GPKGs (not stored in GitHub or Zenodo; generated locally if needed):  
  `2_modelling/02_application/predictions/Filtered_80pct_allattributes/{country}_rf_preds_filtered80.gpkg`
- Summary CSV:  
  `2_modelling/02_application/predictions/Filtered_80pct_allattributes/filtered80_new_summary.csv`


# Imports and paths

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# --- Paths (relative to 2_modelling/02_application) ---
in_folder = Path("predictions")
out_folder = in_folder / "Filtered_80pct_allattributes"
out_folder.mkdir(parents=True, exist_ok=True)

print("Input predictions folder:", in_folder.resolve())
print("Output filtered folder:", out_folder.resolve())


# Main loop (filter by 80% valid blocks)

In [ ]:
# ===============================================================
# Filter cities with ≥80% valid blocks and SAVE ALL ORIGINAL ATTRS
# ===============================================================

summary_records = []

for gpkg_file in in_folder.glob("*_rf_preds.gpkg"):
    country = gpkg_file.stem.replace("_rf_preds", "")
    print(f"\nProcessing {country}...")

    try:
        # Load ALL columns to preserve every attribute in the output
        gdf = gpd.read_file(gpkg_file)

        # Ensure required columns exist
        required = {"REG1_GHSL", "UC_NM_MN", "rf_label", "POP_SEG", "geometry"}
        missing = [c for c in required if c not in gdf.columns]
        if missing:
            raise ValueError(f"Missing required columns: {missing}")

    except Exception as e:
        print(f"⚠️ Skipping {country}: {e}")
        continue

    # --- Total segments before filtering ---
    total_segments = len(gdf)

    # --- Coerce rf_label to numeric if needed (e.g., '1'/'0' strings) ---
    gdf["rf_label"] = pd.to_numeric(gdf["rf_label"], errors="coerce")

    # --- Population before filtering ---
    total_pop_before = gdf["POP_SEG"].sum(skipna=True)
    deprived_pop_before = gdf.loc[gdf["rf_label"] == 1, "POP_SEG"].sum(skipna=True)

    # --- Mark valid blocks (both rf_label and POP_SEG present) ---
    gdf["valid"] = ~(gdf["rf_label"].isna() | gdf["POP_SEG"].isna())

    # --- Compute city-level validity ---
    city_stats = (
        gdf.groupby("UC_NM_MN")["valid"]
        .agg(["sum", "count"])
        .rename(columns={"sum": "ValidBlocks", "count": "TotalBlocks"})
        .reset_index()
    )
    city_stats["PctValid"] = (city_stats["ValidBlocks"] / city_stats["TotalBlocks"]) * 100

    # --- Determine valid vs discarded cities (≥80% valid blocks) ---
    valid_cities = city_stats.loc[city_stats["PctValid"] >= 80, "UC_NM_MN"].tolist()
    discarded_cities = city_stats.loc[city_stats["PctValid"] < 80, "UC_NM_MN"].tolist()

    # --- Filter data (keep all attributes, only valid blocks in valid cities) ---
    filtered = gdf[gdf["UC_NM_MN"].isin(valid_cities) & gdf["valid"]].copy()

    # --- Stats after filtering ---
    segments_after = len(filtered)
    segments_discarded = total_segments - segments_after

    total_pop_after = filtered["POP_SEG"].sum(skipna=True)
    deprived_pop_after = filtered.loc[filtered["rf_label"] == 1, "POP_SEG"].sum(skipna=True)

    # --- Save filtered file (only if it contains data) ---
    if not filtered.empty:
        out_path = out_folder / f"{country}_rf_preds_filtered80.gpkg"

        # Drop helper column if you prefer not to keep it in the output
        filtered = filtered.drop(columns=["valid"], errors="ignore")

        filtered.to_file(out_path, driver="GPKG")
        print(
            f"✅ Saved filtered file for {country} "
            f"({len(valid_cities)} cities, {segments_after} segments)."
        )
    else:
        print(f"⚠️ No valid data retained for {country}.")

    # --- Append summary record ---
    summary_records.append({
        "Country": country,
        "ValidCities": len(valid_cities),
        "DiscardedCities": len(discarded_cities),
        "SegmentsBefore": total_segments,
        "SegmentsAfter": segments_after,
        "SegmentsDiscarded": segments_discarded,
        "TotalPopBefore": total_pop_before,
        "TotalPopAfter": total_pop_after,
        "DeprivedPopBefore": deprived_pop_before,
        "DeprivedPopAfter": deprived_pop_after
    })

# --- Write summary CSV ---
if summary_records:
    summary_df = pd.DataFrame(summary_records)
    summary_csv = out_folder / "filtered80_new_summary.csv"
    summary_df.to_csv(summary_csv, index=False)
    print(f"\n📄 Summary written to: {summary_csv}")
else:
    print("\nℹ️ No files processed successfully; no summary generated.")
